# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [ ]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00196447  1.4050362  -0.19899885 -0.26150286  0.00228316  0.04507622
  0.          0.        ] -> Action: 3
Obs: [-3.8412095e-03  1.3985773e+00 -1.8768463e-01 -2.8706375e-01
  2.2952559e-03  2.4261718e-04  0.0000000e+00  0.0000000e+00] -> Action: 3
Obs: [-5.6324005e-03  1.3915226e+00 -1.7694189e-01 -3.1354532e-01
  1.5292098e-04 -4.2850763e-02  0.0000000e+00  0.0000000e+00] -> Action: 1
Obs: [-7.5176237e-03  1.3838642e+00 -1.8876223e-01 -3.4037489e-01
  3.8314995e-04  4.6050069e-03  0.0000000e+00  0.0000000e+00] -> Action: 2
Obs: [-9.3731880e-03  1.3768969e+00 -1.8592905e-01 -3.0965564e-01
  7.5592106e-04  7.4563213e-03  0.0000000e+00  0.0000000e+00] -> Action: 1
Obs: [-0.01130075  1.3693386  -0.19496705 -0.3359238   0.00294098  0.0437052
  0.          0.        ] -> Action: 2
Obs: [-0.01332207  1.3620607  -0.20390408 -0.3234741   0.00468567  0.0348969
  0.          0.        ] -> Action: 3
Obs: [-0.01526289  1.3541859  -0.19379798 -0.34998688  0.00440348 -0.00564421
  0.     

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [ ]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [ ]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00769272  1.4063972  -0.77920526 -0.2010456   0.00892075  0.1765017
  0.          0.        ] -> Action: 0
Obs: [-0.0153861   1.401298   -0.77815163 -0.22670616  0.01764233  0.17444886
  0.          0.        ] -> Action: 0
Obs: [-0.02307968  1.3955997  -0.778179   -0.2533884   0.02635878  0.17434517
  0.          0.        ] -> Action: 0
Obs: [-0.03077364  1.3893021  -0.77820474 -0.2800658   0.03507411  0.1743226
  0.          0.        ] -> Action: 0
Obs: [-0.03846779  1.3824058  -0.77823013 -0.3067402   0.04378801  0.17429459
  0.          0.        ] -> Action: 2
Obs: [-0.04631567  1.3755926  -0.79294986 -0.30306834  0.0518628   0.16151063
  0.          0.        ] -> Action: 2
Obs: [-0.05430641  1.3694907  -0.80675614 -0.27147627  0.05947294  0.15221705
  0.          0.        ] -> Action: 0
Obs: [-0.06229734  1.3627895  -0.80677783 -0.29815716  0.06708203  0.15219577
  0.          0.        ] -> Action: 0
Obs: [-0.07028856  1.355489   -0.80680025 -0.3248257   0.07468971 

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [ ]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
import gymnasium as gym
import numpy as np
from enum import Enum

# Định nghĩa action và observation
class Act(Enum):
    NO_OP = 0
    LEFT = 1
    MAIN = 2
    RIGHT = 3

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7

# Reflex-based agent
def rocket_agent_function(obs):
    vy = obs[Obs.VY.value]
    angle = obs[Obs.ANGLE.value]
    left_leg = obs[Obs.LEFT_LEG_CONTACT.value]
    right_leg = obs[Obs.RIGHT_LEG_CONTACT.value]

    # Nếu đã tiếp đất thì không làm gì
    if left_leg or right_leg:
        return Act.NO_OP.value

    # Nếu rơi quá nhanh thì bật động cơ chính
    if vy < -0.3:
        return Act.MAIN.value

    # Nếu lệch sang trái quá nhiều thì bật động cơ phải
    if angle < -0.1:
        return Act.RIGHT.value

    # Nếu lệch sang phải quá nhiều thì bật động cơ trái
    if angle > 0.1:
        return Act.LEFT.value

    return Act.NO_OP.value

# Hàm chạy mô phỏng
def run_episode(agent_function, max_steps=1000):
    env = gym.make("LunarLander-v3", render_mode="human")
    obs, info = env.reset(seed=42)

    for step in range(max_steps):
        action = agent_function(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        env.render()

        if terminated:
            print(f"Thử nghiệm kết thúc sau {step} bước. Kết quả cuối cùng: {reward}")
            break

    env.close()

# Thử chạy
run_episode(rocket_agent_function)


Thử nghiệm kết thúc sau 109 bước. Kết quả cuối cùng: -100
